In [ ]:
def eval_DNSMOS(dir_in,csv_out,sr):
    models = glob.glob(os.path.join(dir_in, "*"))
    audio_clips_list = []

    primary_model_path = os.path.join("src",'DNSMOS', 'sig_bak_ovr.onnx')

    compute_score = ComputeScore(primary_model_path,sr)

    rows = []
    clips = []
    clips = glob.glob(os.path.join(dir_in, "*.wav"))
    for m in tqdm(models):
        max_recursion_depth = 10
        audio_path = os.path.join(dir_in, m)
        audio_clips_list = glob.glob(os.path.join(audio_path, "*.wav"))
        while len(audio_clips_list) == 0 and max_recursion_depth > 0:
            audio_path = os.path.join(audio_path, "**")
            audio_clips_list = glob.glob(os.path.join(audio_path, "*.wav"))
            max_recursion_depth -= 1
        clips.extend(audio_clips_list)

    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_url = {executor.submit(compute_score, clip): clip for clip in clips}
        for future in tqdm(concurrent.futures.as_completed(future_to_url)):
            clip = future_to_url[future]
            try:
                data = future.result()
            except Exception as exc:
                print('%r generated an exception: %s' % (clip, exc))
            else:
                rows.append(data)            

    df = pd.DataFrame(rows)
    csv_path = csv_out
    df.to_csv(csv_path)